## Tuning YAHPO using `HPBandSter`

In order to demonstrate using YAHPO Gym more in-depth we provide a full example benchmarking `HPBandSter` on an `lcbench` task.

Note, that this requires the `hpbandster` module to be installed.

We again start by importing the relevant modules:

In [1]:
from yahpo_gym import *
import time
import numpy as np

First, we define a worker class as required by `HPBandSter` that internally calls our `objective_function`.

In [2]:
from hpbandster.core.worker import Worker
import hpbandster.core.nameserver as hpns
from hpbandster.optimizers import BOHB as BOHB

class lcbench(Worker):

    def __init__(self, *args, sleep_interval=0, **kwargs):
        super().__init__(*args, **kwargs)
        self.bench = bench
        self.sleep_interval = sleep_interval

    def compute(self, config, budget, **kwargs):
        """
        Args:
            config: dictionary containing the sampled configurations by the optimizer
            budget: (float) amount of epochs the model can use to train

        Returns:
            dictionary with mandatory fields:
                "loss" (scalar)
                "info" (dict)
        """

        config.update({"epoch": int(np.round(budget))})  # update epoch
        result = bench.objective_function(config)  # evaluate

        time.sleep(self.sleep_interval)

        return({
                    "loss": - float(result[0].get("val_accuracy")),  # we want to maximize validation accuracy
                    "info": "empty"
                })
    
    @staticmethod
    def get_configspace():
        # remove the epoch fidelity parameter for HPBandSter
        cs = bench.get_opt_space(drop_fidelity_params = True)
        return(cs)

Using this worker class, we can now run the full benchmark:

In [3]:
# Initialize the set providing a scenario
bench = benchmark_set.BenchmarkSet("lcbench", instance = "3945")

# Start up the nameserver
NS = hpns.NameServer(run_id="lcbench", host="127.0.0.1", port=None)
NS.start()

# Run BOHB
w = lcbench(sleep_interval=0, nameserver="127.0.0.1", run_id ="lcbench")
w.run(background=True)

bohb = BOHB(configspace=w.get_configspace(),
            run_id="lcbench", nameserver="127.0.0.1",
            min_budget=1, max_budget=52)

res = bohb.run(n_iterations=1)

bohb.shutdown(shutdown_workers=True)
NS.shutdown()

22:41:42 wait_for_workers trying to get the condition
22:41:42 DISPATCHER: started the 'discover_worker' thread
22:41:42 WORKER: Connected to nameserver <Pyro4.core.Proxy at 0x7fd8ff0f4ca0; connected IPv4; for PYRO:Pyro.NameServer@127.0.0.1:9090>
22:41:42 DISPATCHER: started the 'job_runner' thread
22:41:42 WORKER: No dispatcher found. Waiting for one to initiate contact.
22:41:42 WORKER: start listening for jobs
22:41:42 DISPATCHER: Pyro daemon running on localhost:35995
22:41:42 DISPATCHER: Starting worker discovery
22:41:42 DISPATCHER: Found 1 potential workers, 0 currently in the pool.
22:41:42 DISPATCHER: discovered new worker, hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:42 HBMASTER: number of workers changed to 1
22:41:42 Enough workers to start this run!
22:41:42 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
22:41:42 adjust_queue_size: lock accquired
22:41:42 HBMASTER: starting run at 1645998102.6593776
22:41:42 HBMASTE

22:41:42 DISPATCHER: Trying to submit another job.
22:41:42 job_callback for (0, 0, 2) got condition
22:41:42 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
22:41:42 Only 3 run(s) for budget 1.925926 available, need more than 10 -> can't build model!
22:41:42 HBMASTER: Trying to run another job!
22:41:42 job_callback for (0, 0, 2) finished
22:41:42 start sampling a new configuration.
22:41:42 done sampling a new configuration.
22:41:42 HBMASTER: schedule new run for iteration 0
22:41:42 HBMASTER: trying submitting job (0, 0, 3) to dispatcher
22:41:42 HBMASTER: submitting job (0, 0, 3) to dispatcher
22:41:42 DISPATCHER: trying to submit job (0, 0, 3)
22:41:42 DISPATCHER: trying to notify the job_runner thread.
22:41:42 HBMASTER: job (0, 0, 3) submitted to dispatcher
22:41:42 DISPATCHER: Trying to submit another job.
22:41:42 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
22:41:42 DISPATCHER: starting job (0, 0, 3) on hpbandster.run_lcbench.worker.julia-Th

22:41:42 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
22:41:42 DISPATCHER: starting job (0, 0, 6) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:42 DISPATCHER: job (0, 0, 6) dispatched on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:42 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
22:41:42 WORKER: start processing job (0, 0, 6)
22:41:42 WORKER: args: ()
22:41:42 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 27, 'learning_rate': 0.0003719635422064396, 'max_dropout': 0.6967449927551415, 'max_units': 319.52726991765013, 'momentum': 0.5988980231313239, 'num_layers': 4, 'weight_decay': 0.040851777907308436}, 'budget': 1.9259259259259258, 'working_directory': '.'}
22:41:42 WORKER: done with job (0, 0, 6), trying to register it.
22:41:42 WORKER: registered result for job (0, 0, 6) with dispatcher
22:41:42 DISPATCHER: job (0, 0, 6) finished
22:41:42 DISPATCHER: register

22:41:43 DISPATCHER: register_result: lock acquired
22:41:43 DISPATCHER: job (0, 0, 9) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648 finished
22:41:43 job_id: (0, 0, 9)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 107, 'learning_rate': 0.0009832785895794137, 'max_dropout': 0.43946977763568096, 'max_units': 341.69298812838235, 'momentum': 0.10260342812902479, 'num_layers': 1, 'weight_decay': 0.056611657439901715}, 'budget': 1.9259259259259258, 'working_directory': '.'}
result: {'loss': -78.3575668334961, 'info': 'empty'}
exception: None

22:41:43 job_callback for (0, 0, 9) started
22:41:43 DISPATCHER: Trying to submit another job.
22:41:43 job_callback for (0, 0, 9) got condition
22:41:43 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
22:41:43 HBMASTER: Trying to run another job!
22:41:43 job_callback for (0, 0, 9) finished
22:41:43 start sampling a new configuration.
22:41:43 done sampling a new configuration.
22:41:4

22:41:43 HBMASTER: submitting job (0, 0, 13) to dispatcher
22:41:43 DISPATCHER: trying to submit job (0, 0, 13)
22:41:43 DISPATCHER: trying to notify the job_runner thread.
22:41:43 HBMASTER: job (0, 0, 13) submitted to dispatcher
22:41:43 DISPATCHER: Trying to submit another job.
22:41:43 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
22:41:43 DISPATCHER: starting job (0, 0, 13) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:43 WORKER: start processing job (0, 0, 13)
22:41:43 DISPATCHER: job (0, 0, 13) dispatched on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:43 WORKER: args: ()
22:41:43 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
22:41:43 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 27, 'learning_rate': 0.00027068249438411796, 'max_dropout': 0.7033797386497097, 'max_units': 84.97659175373788, 'momentum': 0.1816475754329191, 'num_layers': 3, 'weight_decay': 0

...

22:41:43 DISPATCHER: trying to notify the job_runner thread.
22:41:43 HBMASTER: job (0, 0, 26) submitted to dispatcher
22:41:43 DISPATCHER: Trying to submit another job.
22:41:43 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
22:41:43 DISPATCHER: starting job (0, 0, 26) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:43 DISPATCHER: job (0, 0, 26) dispatched on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:43 WORKER: start processing job (0, 0, 26)
22:41:43 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
22:41:43 WORKER: args: ()
22:41:43 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 20, 'learning_rate': 0.01578585227231788, 'max_dropout': 0.08689066479089891, 'max_units': 101.43842113193914, 'momentum': 0.2508703187873067, 'num_layers': 3, 'weight_decay': 0.016760376232278593}, 'budget': 1.9259259259259258, 'working_directory': '.'}
22:41:43 WORKER: done with job (0, 

22:41:44 WORKER: start processing job (0, 0, 16)
22:41:44 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
22:41:44 WORKER: args: ()
22:41:44 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 18, 'learning_rate': 0.08192954284369308, 'max_dropout': 0.9674905610949995, 'max_units': 86.0535138745652, 'momentum': 0.3585973064115494, 'num_layers': 3, 'weight_decay': 0.09231436559746088}, 'budget': 5.777777777777778, 'working_directory': '.'}
22:41:44 WORKER: done with job (0, 0, 16), trying to register it.
22:41:44 WORKER: registered result for job (0, 0, 16) with dispatcher
22:41:44 DISPATCHER: job (0, 0, 16) finished
22:41:44 DISPATCHER: register_result: lock acquired
22:41:44 DISPATCHER: job (0, 0, 16) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648 finished
22:41:44 job_id: (0, 0, 16)
kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 18, 'learning_rate': 0.08192954284369308, 'max_dropout': 0.9674905610949995,

22:41:44 job_callback for (0, 0, 21) started
22:41:44 DISPATCHER: Trying to submit another job.
22:41:44 job_callback for (0, 0, 21) got condition
22:41:44 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
22:41:44 Only 6 run(s) for budget 5.777778 available, need more than 10 -> can't build model!
22:41:44 HBMASTER: Trying to run another job!
22:41:44 job_callback for (0, 0, 21) finished
22:41:44 HBMASTER: schedule new run for iteration 0
22:41:44 HBMASTER: trying submitting job (0, 0, 23) to dispatcher
22:41:44 HBMASTER: submitting job (0, 0, 23) to dispatcher
22:41:44 DISPATCHER: trying to submit job (0, 0, 23)
22:41:44 DISPATCHER: trying to notify the job_runner thread.
22:41:44 HBMASTER: job (0, 0, 23) submitted to dispatcher
22:41:44 DISPATCHER: Trying to submit another job.
22:41:44 HBMASTER: running jobs: 1, queue sizes: (0, 1) -> wait
22:41:44 DISPATCHER: starting job (0, 0, 23) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22

22:41:44 DISPATCHER: job (0, 0, 16) dispatched on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648
22:41:44 WORKER: start processing job (0, 0, 16)
22:41:44 DISPATCHER: jobs to submit = 0, number of idle workers = 0 -> waiting!
22:41:44 WORKER: args: ()
22:41:44 WORKER: kwargs: {'config': {'OpenML_task_id': '3945', 'batch_size': 18, 'learning_rate': 0.08192954284369308, 'max_dropout': 0.9674905610949995, 'max_units': 86.0535138745652, 'momentum': 0.3585973064115494, 'num_layers': 3, 'weight_decay': 0.09231436559746088}, 'budget': 17.333333333333332, 'working_directory': '.'}
22:41:44 WORKER: done with job (0, 0, 16), trying to register it.
22:41:44 WORKER: registered result for job (0, 0, 16) with dispatcher
22:41:44 DISPATCHER: job (0, 0, 16) finished
22:41:44 DISPATCHER: register_result: lock acquired
22:41:44 DISPATCHER: job (0, 0, 16) on hpbandster.run_lcbench.worker.julia-ThinkPad-T490s.237975140571618211648 finished
22:41:44 job_id: (0, 0, 16)
kwargs: {'co

22:41:44 job_callback for (0, 0, 19) started
22:41:44 DISPATCHER: Trying to submit another job.
22:41:44 job_callback for (0, 0, 19) got condition
22:41:44 DISPATCHER: jobs to submit = 0, number of idle workers = 1 -> waiting!
22:41:44 Only 1 run(s) for budget 52.000000 available, need more than 10 -> can't build model!
22:41:44 HBMASTER: Trying to run another job!
22:41:44 job_callback for (0, 0, 19) finished
22:41:44 HBMASTER: shutdown initiated, shutdown_workers = True
22:41:44 WORKER: shutting down now!
22:41:44 DISPATCHER: Dispatcher shutting down
22:41:44 DISPATCHER: discover_workers shutting down
22:41:44 DISPATCHER: 'discover_worker' thread exited
22:41:44 DISPATCHER: Trying to submit another job.
22:41:44 DISPATCHER: job_runner shutting down
22:41:44 DISPATCHER: 'job_runner' thread exited
22:41:44 DISPATCHER: shut down complete


Then we obtain the final results:

In [4]:
id2config = res.get_id2config_mapping()
id2config

{(0,
  0,
  0): {'config': {'OpenML_task_id': '3945',
   'batch_size': 159,
   'learning_rate': 0.0006171073605017066,
   'max_dropout': 0.2130308111610304,
   'max_units': 122.27552565196628,
   'momentum': 0.3974330219838885,
   'num_layers': 3,
   'weight_decay': 0.048280502515210044}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  1): {'config': {'OpenML_task_id': '3945',
   'batch_size': 180,
   'learning_rate': 0.00038511728912622283,
   'max_dropout': 0.08054513105583727,
   'max_units': 506.9112674242038,
   'momentum': 0.4132324409889284,
   'num_layers': 1,
   'weight_decay': 0.09731516069386124}, 'config_info': {'model_based_pick': False}},
 (0,
  0,
  2): {'config': {'OpenML_task_id': '3945',
   'batch_size': 344,
   'learning_rate': 0.03662645469351678,
   'max_dropout': 0.06905644350224871,
   'max_units': 104.18754868431009,
   'momentum': 0.560677485741364,
   'num_layers': 3,
   'weight_decay': 0.04853358503559112}, 'config_info': {'model_based_pick': False}}

In [5]:
incumbent = res.get_incumbent_id()
incumbent

(0, 0, 19)

In [6]:

print("Best found configuration:", id2config[incumbent]["config"])
print("A total of %i unique configurations where sampled." % len(id2config.keys()))
print("A total of %i runs where executed." % len(res.get_all_runs()))
print("Total budget corresponds to %.1f full function evaluations."%(sum([r.budget for r in res.get_all_runs()])/1))

Best found configuration: {'OpenML_task_id': '3945', 'batch_size': 19, 'learning_rate': 0.09620839886979782, 'max_dropout': 0.7616426039620794, 'max_units': 93.37841645498287, 'momentum': 0.4044561710909874, 'num_layers': 1, 'weight_decay': 0.06780310912777217}
A total of 27 unique configurations where sampled.
A total of 40 runs where executed.
Total budget corresponds to 208.0 full function evaluations.
